<a href="https://colab.research.google.com/github/youngduck98/algorithm_study_in_cau/blob/master/Semi_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/machine_learning projectr(4-1 final)/
"""

Mounted at /content/drive
/content/drive/MyDrive/machine_learning projectr(4-1 final)


In [ ]:
#!unzip "/content/drive/MyDrive/machine_learning projectr(4-1 final)/ML_Challenge_Data_Codes.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: data/Supervised_Learning/labeled/5/5_7127.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7150.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7179.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7200.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7219.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_723.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7252.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7266.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7275.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7285.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7288.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_73.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_730.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7312.JPEG  
  inflating: data/Supervised_Learning/labeled/5/5_7314.JPEG  
  inflating: data/Supervised_Learning/

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision import models

import os
from PIL import Image
import argparse

import torch.nn.functional as F

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.classes = os.listdir(root)
        self.class_to_idx = {c: int(c) for i, c in enumerate(self.classes)}
        self.imgs = []
        for c in self.classes:
            class_dir = os.path.join(root, c)
            for filename in os.listdir(class_dir):
                path = os.path.join(class_dir, filename)
                self.imgs.append((path, self.class_to_idx[c]))
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, index):
        path, target = self.imgs[index]
        img = Image.open(path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, target

In [ ]:
class CustomDataset_Nolabel(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        ImageList = os.listdir(root)
        self.imgs = []
        for filename in ImageList:
            path = os.path.join(root, filename)
            self.imgs.append(path)
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, index):
        path = self.imgs[index]
        img = Image.open(path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img

In [ ]:
####################
#If you want to use your own custom model
#Write your code here
####################
class Custom_model(nn.Module):
    def __init__(self):
        super(Custom_model, self).__init__()
        #place your layers
        #CNN, MLP and etc.

    def forward(self, input):
        #place for your model
        #Input: 3* Width * Height
        #Output: Probability of 10 class label
        return predicted_label


class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

In [ ]:
####################
#Modify your code here
####################
def model_selection(selection):
    if selection == "resnet":
        model = models.resnet18(pretrained=True)
        model.conv1 =  nn.Conv2d(3, 64, kernel_size=3,stride=1, padding=1, bias=False)
        model.layer4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True))
        model.fc = nn.Linear(512, 10)
    elif selection == "vgg":
        model = models.vgg11_bn(pretrained=True)
        model.features = nn.Sequential(*list(model.features.children())[:-7])
        model.classifier = nn.Sequential( nn.Linear(in_features=25088, out_features=10, bias=True))
    elif selection == "mobilenet":
        model = models.mobilenet_v2()
        model.classifier = nn.Sequential( nn.Linear(in_features=1280, out_features=10, bias=True))
    elif  selection =='custom':
        model = Custom_model()
    return model

In [ ]:
def cotrain(net1,net2, labeled_loader, unlabled_loader, optimizer1_1, optimizer1_2, optimizer2_1, optimizer2_2, criterion):
    #The inputs are as below.
    #{First model, Second model, Loader for labeled dataset with labels, Loader for unlabeled dataset that comes without any labels,
    global cotrain_start
    global cotrain_idx
    global percent_1
    global percent_2
    net1.train()
    net2.train()
    total_loss_1 = 0
    total_loss_2 = 0
    total_loss = 0
    train_loss_1 = 0
    train_loss_2 = 0
    train_loss = 0
    correct_1 = 0
    correct_2 = 0
    correct = 0
    total = 0
    k = 0.8

    if cotrain_start > cotrain_idx or cotrain_idx % 6 == 0:
    #labeled_training
      for batch_idx, (inputs, targets) in enumerate(labeled_loader):
          if torch.cuda.is_available():
              inputs, targets = inputs.cuda(), targets.cuda()
          optimizer1_1.zero_grad()
          optimizer2_1.zero_grad()
  ####################
  #Add your code here
  ####################
          outputs_1 = net1(inputs)
          outputs_2 = net2(inputs)

          loss_1 = criterion(outputs_1, targets)
          loss_2 = criterion(outputs_2, targets)

          loss_1.backward()
          loss_2.backward()

          optimizer1_1.step()
          optimizer2_1.step()

          total_loss_1 += loss_1.item()
          total_loss_2 += loss_2.item()

          x, predicted_1 = outputs_1.max(1)
          _, predicted_2 = outputs_2.max(1)

          correct_1 += predicted_1.eq(targets).sum().item()
          correct_2 += predicted_2.eq(targets).sum().item()

          total += targets.size(0)

          percent_1 = correct_1/total
          percent_2 = correct_2/total

          if(batch_idx == 0):
            cotrain_idx += 1
            print("idx:", cotrain_idx)

          if batch_idx % 30 == 0:
              print('model1: Batch {}/{} Loss: {:.4f} | Acc: {:.2f}%'.format(
                  batch_idx, len(labeled_loader), total_loss_1/(batch_idx+1), 100.*correct_1/total))

          if batch_idx % 30 == 0:
              print('model2: Batch {}/{} Loss: {:.4f} | Acc: {:.2f}%'.format(
                  batch_idx, len(labeled_loader), total_loss_2/(batch_idx+1), 100.*correct_2/total))

    if cotrain_start <= cotrain_idx:
    #unlabeled_training
      for batch_idx, inputs in enumerate(unlabled_loader):
          if torch.cuda.is_available():
              inputs = inputs.cuda()
          optimizer1_2.zero_grad()
          optimizer2_2.zero_grad()

          if(batch_idx == 0):
            cotrain_idx += 1
            print("idx:", cotrain_idx)

          net1.eval()
          outputs_1 = net1(inputs)
          values_1, predicted_1 = outputs_1.max(1)
          probs, _ = F.softmax(outputs_1, dim=1).max(1)

          greater_index_1 = []
          for i in range(len(probs)):
            if(probs[i] >= 0.97):
              greater_index_1.append(True)
            else:
              greater_index_1.append(False)

          new_inputs_1 = inputs[greater_index_1]
          _, new_target_1 = net1(new_inputs_1).max(1)

          net2.train()
          new_outputs_2 = net2(new_inputs_1)
          _, test_index = net1(new_inputs_1).max(1)
          if(not torch.equal(new_target_1, test_index)):
            print(test_index)
            print(predicted_1[greater_index_1])
            print(new_target_1)
            print("error")
            return
          loss_2 = criterion(new_outputs_2, new_target_1)
          loss_2.backward()
          optimizer2_2.step()

          net2.eval()
          outputs_2 = net2(inputs)
          values_2, predicted_2 = outputs_2.max(1)
          probs2, _ = F.softmax(outputs_2, dim=1).max(1)

          greater_index_2 = []
          for i in range(len(probs)):
            if(probs2[i] >= 0.97):
              greater_index_2.append(True)
            else:
              greater_index_2.append(False)

          new_inputs_2 = inputs[greater_index_2]
          _, new_target_2 = net2(new_inputs_2).max(1)

          net1.train()
          new_output_1 = net1(new_inputs_2)
          _, test_index = new_output_1.max(1)

          if(not torch.equal(new_target_2, predicted_2[greater_index_2])):
            print(test_index)
            print(predicted_2[greater_index_1])
            print(new_target_2)
            print("error")
            return

          loss_1 = criterion(new_output_1, new_target_2)
          loss_1.backward()
          optimizer1_2.step()

"""

      else:
        pass
####################
#Add your code here
####################

        #hint :
        #agree = predicted1 == predicted2
        #pseudo_labels from agree
"""
def test(net, testloader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            if torch.cuda.is_available():
                inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        return 100. * correct / total

In [ ]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument('--test',  type=str,  default='False')
    parser.add_argument('--student_abs_path',  type=str,  default='./')
    parser.add_argument('-f')
    args = parser.parse_args()

    batch_size =  32

    if args.test == 'False':
        train_transform = transforms.Compose([
                    transforms.RandomResizedCrop(64, scale=(0.2, 1.0)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])
        test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])

        dataset = CustomDataset(root = './data/Semi-Supervised_Learning/labeled', transform = train_transform)
        labeled_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

        dataset = CustomDataset_Nolabel(root = './data/Semi-Supervised_Learning/unlabeled', transform = train_transform)
        unlabeled_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

        dataset = CustomDataset(root = './data/Semi-Supervised_Learning/val', transform = test_transform)
        val_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    else :
        test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    if not os.path.exists(os.path.join(args.student_abs_path, 'logs', 'Semi-Supervised_Learning')):
        os.makedirs(os.path.join(args.student_abs_path, 'logs', 'Semi-Supervised_Learning'))

    model_sel_1 =  'resnet'
    model_sel_2 =  'vgg'

    model1 = model_selection(model_sel_1)
    model2 = model_selection(model_sel_2)

    params_1 = sum(p.numel() for p in model1.parameters() if p.requires_grad) / 1e6
    params_2 = sum(p.numel() for p in model2.parameters() if p.requires_grad) / 1e6

    if torch.cuda.is_available():
        model1 = model1.cuda()
    if torch.cuda.is_available():
        model2 = model2.cuda()

    #You may want to write a loader code that loads the model state to continue the learning process
    #Since this learning process may take a while.


    if torch.cuda.is_available():
        criterion = nn.CrossEntropyLoss().cuda()
    else :
        criterion = nn.CrossEntropyLoss()

    learning_rate_1 = 0.0002
    learning_rate_2 = 0.00002

    optimizer1_1 = optim.Adam(model1.parameters(), lr=learning_rate_1)
    optimizer2_1 = optim.Adam(model2.parameters(), lr=learning_rate_1)

    optimizer1_2 = optim.Adam(model1.parameters(), lr=learning_rate_2)
    optimizer2_2 = optim.Adam(model2.parameters(), lr=learning_rate_2)

    #Input the number of epochs
    epoch = 120
    cotrain_start = 40
    cotrain_idx = 0

    if args.test == 'False':
        assert params_1 < 7.0, "Exceed the limit on the number of model_1 parameters"
        assert params_2 < 7.0, "Exceed the limit on the number of model_2 parameters"

        best_result_1 = 0
        best_result_2 = 0
        for e in range(0, epoch):
            cotrain(model1, model2, labeled_loader, unlabeled_loader, optimizer1_1, optimizer1_2, optimizer2_1, optimizer2_2, criterion)
            if(e == int(cotrain_start/2)):
              optimizer1_1.param_groups[0]['lr'] /=2
              optimizer2_1.param_groups[0]['lr'] /=2
            elif(e == epoch - 30):
              optimizer1_2.param_groups[0]['lr'] /=2
              optimizer2_2.param_groups[0]['lr'] /=2
            tmp_res_1 = test(model1, val_loader)
            # You can change the saving strategy, but you can't change file name/path for each model
            print ("[{}th epoch, model_1] ACC : {}".format(e, tmp_res_1))
            if best_result_1 < tmp_res_1:
                best_result_1 = tmp_res_1
                torch.save(model1.state_dict(),  os.path.join('./logs', 'Semi-Supervised_Learning', 'best_model_1.pt'))

            tmp_res_2 = test(model2, val_loader)
            # You can change save strategy, but you can't change file name/path for each model
            print ("[{}th epoch, model_2] ACC : {}".format(e, tmp_res_2))
            if best_result_2 < tmp_res_2:
                best_result_2 = tmp_res_2
                torch.save(model2.state_dict(),  os.path.join('./logs', 'Semi-Supervised_Learning', 'best_model_2.pt'))
        print('Final performance {} - {}  // {} - {}', best_result_1, params_1, best_result_2, params_2)


    else:
        dataset = CustomDataset(root = '/data/23_1_ML_challenge/Semi-Supervised_Learning/test', transform = test_transform)
        test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

        model1.load_state_dict(torch.load(os.path.join(args.student_abs_path, 'logs', 'Semi-Supervised_Learning', 'best_model_1.pt'), map_location=torch.device('cuda')))
        res1 = test(model1, test_loader)

        model2.load_state_dict(torch.load(os.path.join(args.student_abs_path, 'logs', 'Semi-Supervised_Learning', 'best_model_2.pt'), map_location=torch.device('cuda')))
        res2 = test(model2, test_loader)

        if res1>res2:
            best_res = res1
            best_params = params_1
        else :
            best_res = res2
            best_params = params_2

        print(best_res, ' - ', best_params)

idx: 1
model1: Batch 0/188 Loss: 2.3111 | Acc: 9.38%
model2: Batch 0/188 Loss: 2.3115 | Acc: 3.12%
model1: Batch 30/188 Loss: 1.7607 | Acc: 39.52%
model2: Batch 30/188 Loss: 1.8136 | Acc: 43.75%
model1: Batch 60/188 Loss: 1.5756 | Acc: 46.26%
model2: Batch 60/188 Loss: 1.4285 | Acc: 57.12%
model1: Batch 90/188 Loss: 1.4389 | Acc: 50.89%
model2: Batch 90/188 Loss: 1.2098 | Acc: 63.46%
model1: Batch 120/188 Loss: 1.3721 | Acc: 53.31%
model2: Batch 120/188 Loss: 1.1004 | Acc: 66.37%
model1: Batch 150/188 Loss: 1.3248 | Acc: 55.34%
model2: Batch 150/188 Loss: 1.0235 | Acc: 68.52%
model1: Batch 180/188 Loss: 1.2720 | Acc: 57.53%
model2: Batch 180/188 Loss: 0.9568 | Acc: 70.49%
[0th epoch, model_1] ACC : 60.6
[0th epoch, model_2] ACC : 75.6
idx: 2
model1: Batch 0/188 Loss: 1.0203 | Acc: 68.75%
model2: Batch 0/188 Loss: 0.3727 | Acc: 84.38%
model1: Batch 30/188 Loss: 0.9619 | Acc: 67.64%
model2: Batch 30/188 Loss: 0.5249 | Acc: 82.26%
model1: Batch 60/188 Loss: 0.9338 | Acc: 68.80%
model2: Ba